In [22]:
import numpy as np
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import pipeline, model_selection, preprocessing, metrics
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegressionCV

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
test.head()

,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,amenities,square_feet,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,9554,"Cozy, 3 minutes to Piccadilly Line",PLEASE CONTACT ME BEFORE BOOKING Homely apartm...,"Hello people, This is a bright, comfortable ro...",PLEASE CONTACT ME BEFORE BOOKING Homely apartm...,none,Details to follow..,NaN,details to follow when i get a chance..,"Kitchen, small dining / smoking room (if you s...",...,"{TV,Internet,Wifi,Kitchen,""Smoking allowed"",Br...",NaN,NaN,7.0,1,15.0,1,strict_14_with_grace_period,t,f
1,11076,The Sanctuary,The room has a double bed and a single foldawa...,This Listing is for The Sanctury The accommoda...,The room has a double bed and a single foldawa...,none,"Ealing Broadway, as short walk from our place ...",NaN,extemely good transport links to central londo...,Huge family kitchen and good wifi,...,"{TV,""Cable TV"",Internet,Wifi,Breakfast,""Pets l...",NaN,NaN,NaN,2,35.0,2,strict_14_with_grace_period,f,f
2,13913,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,For art lovers I can give guest my Tate Member...,The flat only a 10 minute walk to Finsbury Par...,Guest will have access to the self catering ki...,...,"{TV,""Cable TV"",Wifi,Kitchen,""Paid parking off ...",538.0,100.0,15.0,1,15.0,1,moderate,f,f
3,17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,"Open from June 2018 after a 3-year break, we a...",Ready again from June 2018 for bookings after ...,"Open from June 2018 after a 3-year break, we a...",none,"Location, location, location! You won't find b...",This property has new flooring throughout. Gue...,You can walk to tourist London or take numerou...,Full use of whole independent apartment,...,"{TV,Wifi,Kitchen,""Paid parking off premises"",E...",NaN,350.0,65.0,4,10.0,3,strict_14_with_grace_period,f,f
4,24328,Battersea 2 bedroom house & parking,"Artist house, high ceiling bedrooms, private p...",- End of terrace two bedroom house close to So...,"Artist house, high ceiling bedrooms, private p...",family,"- Battersea is a quiet family area, easy acces...",- Please have a profile or tell us more about ...,"- 5 mins walk to Battersea Park, 15 mins walk ...",- there is a communal garden in our complex - ...,...,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Free par...",1001.0,250.0,70.0,2,15.0,90,strict_14_with_grace_period,t,t


In [4]:
long_text_cols = ['name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit',
                 'access', 'interaction', 'house_rules', 'host_about', 'amenities']

In [7]:
train_X, valid_X = model_selection.train_test_split(train, test_size=0.3, random_state=42)

In [8]:
for col in long_text_cols:
    train_X[col].fillna('', inplace=True)
    valid_X[col].fillna('', inplace=True)
    test[col].fillna('', inplace=True)

/Users/ponomarevandrew/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [9]:
train_X['all_text'] = train_X[long_text_cols].apply(lambda s: ' '.join(s), axis=1)
valid_X['all_text'] = valid_X[long_text_cols].apply(lambda s: ' '.join(s), axis=1)
test['all_text'] = test[long_text_cols].apply(lambda s: ' '.join(s), axis=1)

<ipython-input-9-4ea0d3d1c9b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X['all_text'] = train_X[long_text_cols].apply(lambda s: ' '.join(s), axis=1)
<ipython-input-9-4ea0d3d1c9b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_X['all_text'] = valid_X[long_text_cols].apply(lambda s: ' '.join(s), axis=1)


In [14]:
stemmer = PorterStemmer()
train_X['cleaned'] = train_X['all_text'].apply(lambda x: " ".join(re.sub("[^a-zA-Z']", " ", x).split()).lower())
#new_train['cleaned'] = new_train['cleaned'].apply(lambda s: " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(s)]))
#new_train['cleaned'] = new_train['cleaned'].apply(lambda x: " ".join([i for i in x.split() if i not in words]))
#new_train['cleaned'] = new_train['cleaned'].apply(lambda x: " ".join([stemmer.stem(i) for i in x.split()]))

<ipython-input-14-87f2ebf533df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X['cleaned'] = train_X['all_text'].apply(lambda x: " ".join(re.sub("[^a-zA-Z']", " ", x).split()).lower())


In [15]:
valid_X['cleaned'] = valid_X['all_text'].apply(lambda x: " ".join(re.sub("[^a-zA-Z']", " ", x).split()).lower())

<ipython-input-15-16757bbf9fa7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_X['cleaned'] = valid_X['all_text'].apply(lambda x: " ".join(re.sub("[^a-zA-Z']", " ", x).split()).lower())


In [16]:
test['cleaned'] = test['all_text'].apply(lambda x: " ".join(re.sub("[^a-zA-Z']", " ", x).split()).lower())

In [53]:
vectorizer = CountVectorizer(min_df = 50, stop_words={'english'})
pipeline = Pipeline([('vect', vectorizer),
#                     ('chi',  SelectKBest(chi2, k=all)),
                     ('clf', LinearRegression())])
pipeline.fit(train_X['cleaned'], train_X['price'])

Pipeline(steps=[('vect', CountVectorizer(min_df=50, stop_words={'english'})),
                ('clf', LinearRegression())])

In [60]:
test['price'] = pipeline.predict(test['cleaned'])

In [64]:
test[['id','price']].to_csv('answer_3.csv', index=False)

In [61]:
test['price'] = test['price'] - test['price'].min()

In [68]:
train_X[train_X['price'] > 100]

,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,cleaning_fee,guests_included,extra_people,minimum_nights,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,price,all_text,cleaned
33454,24754494,Dashing 1BR in Bayswater by Sonder,"At this Sonder, you'll love the chic decor, st...",Every booking is instantly confirmed. Every ca...,"At this Sonder, you'll love the chic decor, st...",none,"Your Sonder is in a beautiful dwelling, conver...","This Sonder does not have air conditioning, bu...",Your Sonder is located at the end of a cul-de-...,,...,63.0,2,5.0,2,strict_14_with_grace_period,f,f,169.0,Dashing 1BR in Bayswater by Sonder At this Son...,dashing br in bayswater by sonder at this sond...
40699,27135434,Family-friendly and Peaceful Base in Old Chelsea,,,,none,,,,,...,NaN,1,0.0,1,flexible,f,f,140.0,Family-friendly and Peaceful Base in Old Chels...,family friendly and peaceful base in old chels...
47199,28752758,Penthouse Three Bedroom Apartment In Kensington,This sophisticated and modern three bedroom pe...,A beautiful and trendy three bedroom penthouse...,This sophisticated and modern three bedroom pe...,none,LOCAL ATTRACTIONS: Opera Holland Park1.5 km Ho...,On request we offer a fantastic TV service whi...,Underground stations which are around: West Ke...,The entire apartment,...,60.0,6,5.0,2,strict_14_with_grace_period,f,f,150.0,Penthouse Three Bedroom Apartment In Kensingto...,penthouse three bedroom apartment in kensingto...
44984,28171829,Perfectly-Located Chelsea Studio Flat,This cosy 1 bedroom flat occupies a perfect lo...,Amenities: - 24/7 guest support - Professiona...,This cosy 1 bedroom flat occupies a perfect lo...,none,"Chelsea: Two long high streets, King's Road a...",,Tube: There are a number of tube stations oper...,Guests will have access to the entire property...,...,60.0,2,15.0,2,moderate,f,f,116.0,Perfectly-Located Chelsea Studio Flat This cos...,perfectly located chelsea studio flat this cos...
20759,21039846,Modern flat in Mayfair - 2 bedrooms & 2 bathrooms,Modern and spacious apartment in the heart of ...,Toiletries: Complimentary toiletries provided:...,Modern and spacious apartment in the heart of ...,none,"Mayfair is a very safe, nice and quiet neighbo...",,You can walk (5-30 minutes) to many of the mai...,,...,NaN,1,0.0,4,strict_14_with_grace_period,f,f,400.0,Modern flat in Mayfair - 2 bedrooms & 2 bathro...,modern flat in mayfair bedrooms bathrooms mode...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21962,21366773,"Beautiful, centrally located 1 bedroom garden ...",Delightful lower ground floor flat with a priv...,"Delightful, cosy and private, the flat compris...",Delightful lower ground floor flat with a priv...,none,The flat is located in a quiet side street off...,,The flat is located within zone 2 on the Londo...,The entire place is yours! Access to the flat ...,...,36.0,2,0.0,4,strict_14_with_grace_period,f,f,110.0,"Beautiful, centrally located 1 bedroom garden ...",beautiful centrally located bedroom garden fla...
6265,16223250,ELEGANT 1840s 2 bedroom apartment by Hyde Park,"Grand 1840s apartment close to Hyde Park, Lanc...",This is a spectacular apartment is on the firs...,"Grand 1840s apartment close to Hyde Park, Lanc...",none,Amazing location with Hyde Park and Oxford str...,This is a unique opportunity to experience liv...,Paddington Station is less than 5 minutes walk...,The whole apartment.,...,100.0,1,0.0,3,flexible,f,f,250.0,ELEGANT 1840s 2 bedroom apartment by Hyde Park...,elegant s bedroom apartment by hyde park grand...
38158,26342100,Goldsborough House - Apartment London SW8,"Spacious three bedroom flat, fantastic locatio...",,"Spacious three bedroom flat, fantastic locatio...",none,,,,,...,NaN,1,0.0,1,flexible,f,f,150.0,Goldsborough House - Apartment London SW8 Spac...,goldsborough house apartment london sw spaciou...
860,13833612,Modern and Luxurious